In [2]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, StratifiedKFold
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.pipeline import Pipeline
import itertools
import pickle
import logging
from logging.handlers import RotatingFileHandler

In [3]:
df = pd.read_csv("../data/run-over-dataset.csv")
print(df.shape)

columns_to_drop = ['VERBALE', 'DATA', 'Tot Testa', 'Tot Torace', 'Tot Addome', 'Tot Scheletro',
                    'Totale', 'Tot Volta cranica', 'Tot Base cranica', 
                    'Tot Neuroc.', 'Tot Splancnoc.', 'Tot Testa',
                    'Tot Tratto toracico', 'Tot Tratto lombare', 'Tot Rachide',
                    ' Totale coste', 'Sterno in toto', 'Tot Bacino', 'I costa dx', 'II costa dx',
                    'III costa dx', 'IV costa dx', 'V costa dx', 'VI costa dx', 'VII costa dx', 
                    'VIII costa dx', 'IX costa dx', 'X costa dx', 'XI costa dx', 'XII costa dx',
                    'I costa sx', 'II costa sx', 'III costa sx', 'IV costa sx', 'V costa sx', 
                    'VI costa sx', 'VII costa sx', 'VIII costa sx', 'IX costa sx', 
                    'X costa sx', 'XI costa sx', 'XII costa sx']

X = df.drop(columns=columns_to_drop)
print(X.shape)

X['ALTEZZA'] = [int(float(h.replace(',', '.'))*100) for h in X['ALTEZZA']]
X['PESO'] = [int(float(str(h).replace(',', '.'))) for h in X['PESO']]
X['BMI'] = [float(str(h).replace(',', '.')) for h in X['BMI']]

num_unique_values = X.nunique()
constant_columns = num_unique_values[num_unique_values == 1].index.tolist()

X = X.drop(columns=constant_columns)
X = X.T.drop_duplicates().T
print(X.shape)

(130, 367)
(130, 326)
(130, 274)


In [3]:
random_seeds = np.random.randint(2343, 3485327, size=5)
random_seeds

array([2456871, 1978223, 1880141, 2329972,  816489])

In [4]:
random_seeds = [2456871, 1978223, 1880141, 2329972,  816489]

In [5]:
def add_record(df, record):
    new_record = pd.DataFrame(record, index=[0])
    df = pd.concat([df, new_record], ignore_index=True)
    return df  

In [5]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

file_handler = logging.FileHandler('if_nested_cv.log')
file_handler.setLevel(logging.INFO)
file_handler.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s - %(message)s'))

logger = logging.getLogger()

logger.handlers = []
logger.addHandler(file_handler)

In [7]:
def nested_cv(X, random_seeds, n_outer_folds=5, n_inner_folds=3, pca_components=[], mod_selection_score=accuracy_score, positive_class=0):
    n_estimatorss = np.arange(50, 201, 50)
    contaminations = np.linspace(0.01, 0.5, 7)
    max_featuress = np.linspace(0.1, 1.0, 5)
    scalers = [StandardScaler(), MinMaxScaler(), RobustScaler()]
    
    best_params = {'n_estimators': 0, 'contamination': 0, 'max_features': 0, 'pca': 0, 'scaler': ''}
    accuracy_scores = []
    precision_scores = []
    recall_scores = []
    f1_scores = []
    best_overall_accuracy = 0
    best_overall_params = {'n_estimators': 0, 'contamination': 0, 'max_features': 0, 'pca': 0, 'scaler': ''}

    y = X['Mezzo'].values
    y = [0. if x==positive_class else 1. for x in y]
    y = np.array(y, dtype=float)
    
    X = X.drop(columns='Mezzo').values

    for seed in random_seeds:
        outer_cv = StratifiedKFold(n_splits=n_outer_folds, shuffle=True, random_state=seed)
        
        for outer_cv_number, (train_idx, test_idx) in enumerate(outer_cv.split(X, y)):
            X_train, X_test = X[train_idx], X[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]

            inner_cv = StratifiedKFold(n_splits=n_inner_folds, shuffle=True, random_state=seed)

            best_score = 0
            best_pipeline = Pipeline([('isolation_forest', IsolationForest())])
            for inner_cv_number, (trainval_idx, valid_idx) in enumerate(inner_cv.split(X_train, y_train)):
                X_trainval, X_valid = X_train[trainval_idx], X_train[valid_idx]
                y_trainval, y_valid = y_train[trainval_idx], y_train[valid_idx]

                idxs_neg = np.where(y_trainval == 1)[0]
                X_valid = np.append(X_valid, X_trainval[idxs_neg], axis=0)
                y_valid = np.append(y_valid, y_trainval[idxs_neg])

                X_trainval = np.delete(X_trainval, idxs_neg, axis=0)
                y_trainval = np.delete(y_trainval, idxs_neg)

                
                for params in itertools.product(n_estimatorss, contaminations, max_featuress, pca_components, scalers):
                    pipeline = Pipeline([
                        ('scaler', params[4]),
                        ('pca', PCA(n_components=params[3])),
                        ('isolation_forest', IsolationForest(n_estimators=params[0], contamination=params[1], max_features=params[2])) 
                    ])
                    
                    pipeline.fit(X_trainval)
                    
                    pred_values = pipeline.predict(X_valid)
                    true_values = [1 if y == 0 else -1 for y in y_valid]
                    
                    score = mod_selection_score(true_values, pred_values)
                    curr_params = {
                            'n_estimators': params[0],
                            'contamination': params[1],
                            'max_features': params[2],
                            'pca': params[3],
                            'scaler': params[4]
                        }
                    logging.info(f"inner cv number: {inner_cv_number}, {mod_selection_score.__name__}: {score}, with params: {curr_params}")
                        
                    if score > best_score:
                        best_score = score
                        best_pipeline = pipeline
                        best_params = curr_params

            idxs_neg = np.where(y_train == 1)[0]
            X_train = np.delete(X_train, idxs_neg, axis=0)

            best_pipeline.fit(X_train)
            pred_values = best_pipeline.predict(X_test)
            true_values = [1 if y == 0 else -1 for y in y_test]

            accuracy = accuracy_score(true_values, pred_values)
            precision = precision_score(true_values, pred_values, zero_division=0.0)
            recall = recall_score(true_values, pred_values)
            f1 = f1_score(true_values, pred_values)

            if accuracy > best_overall_accuracy:
                best_overall_accuracy = accuracy_score(true_values, pred_values)
                best_overall_params = best_params

            logging.info(f"outer cv number: {outer_cv_number}, accuracy: {score}, precision: {precision}, recall: {recall}, f1: {f1} with params: {curr_params}")

            accuracy_scores.append(accuracy)
            precision_scores.append(precision)
            recall_scores.append(recall)
            f1_scores.append(f1)

    return {'algorythm': 'IsolationForest',
            'best n_estimators': best_overall_params['n_estimators'],
            'best contamination': best_overall_params['contamination'],
            'best max_features': best_overall_params['max_features'],
            'best pca components': best_overall_params['pca'],
            'best scaler': best_overall_params['scaler'],
            'score used for model selection': mod_selection_score.__name__,
            'method used for model selection': 'nested cv',
            'accuracy mean': np.mean(accuracy_scores) * 100,
            'accuracy std': np.std(accuracy_scores) * 100,
            'precision mean': np.mean(precision_scores) * 100,
            'precision  std': np.std(precision_scores) * 100,
            'recall mean': np.mean(recall_scores) * 100,
            'recall std': np.std(recall_scores) * 100,
            'f1 mean': np.mean(f1_scores) * 100,
            'f1 std': np.std(f1_scores) * 100,
            'best overall accuracy': best_overall_accuracy * 100,
            'class': positive_class}

In [9]:
results = nested_cv(X, random_seeds[0:1], pca_components=[20,28,35], mod_selection_score=accuracy_score, positive_class=0)
scores_df = pd.DataFrame(results, index=[0])
scores_df

,algorythm,best n_estimators,best contamination,best max_features,best pca components,best scaler,score used for model selection,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,f1 mean,f1 std,best overall accuracy,class
0,IsolationForest,50,0.5,1.0,35,StandardScaler(),accuracy_score,nested cv,63.846154,7.133553,62.919677,5.075405,80.0,8.329931,70.365347,6.093081,76.923077,0


In [11]:
scores_df = add_record(scores_df, nested_cv(X, random_seeds[0:1], pca_components=[20,28,35], mod_selection_score=f1_score, positive_class=0))
scores_df

,algorythm,best n_estimators,best contamination,best max_features,best pca components,best scaler,score used for model selection,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,f1 mean,f1 std,best overall accuracy,class
0,IsolationForest,50,0.5,1.0,35,StandardScaler(),accuracy_score,nested cv,63.846154,7.133553,62.919677,5.075405,80.0,8.329931,70.365347,6.093081,76.923077,0
1,IsolationForest,50,0.5,1.0,28,RobustScaler(),f1_score,nested cv,66.923077,5.217177,64.777274,5.837249,90.0,10.690450,74.557812,2.470739,73.076923,0


In [12]:
file_path = 'if_exp5_df.pickle'

with open(file_path, 'wb') as file:
    pickle.dump(scores_df, file)

In [7]:
file_path = 'if_exp5_df.pickle'

with open(file_path, 'rb') as file:
    scores_df = pickle.load(file)

scores_df

,algorythm,best n_estimators,best contamination,best max_features,best pca components,best scaler,score used for model selection,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,f1 mean,f1 std,best overall accuracy,class
0,IsolationForest,50,0.5,1.0,35,StandardScaler(),accuracy_score,nested cv,63.846154,7.133553,62.919677,5.075405,80.0,8.329931,70.365347,6.093081,76.923077,0
1,IsolationForest,50,0.5,1.0,28,RobustScaler(),f1_score,nested cv,66.923077,5.217177,64.777274,5.837249,90.0,10.690450,74.557812,2.470739,73.076923,0


In [8]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

file_handler = logging.FileHandler('if_nested_cv_100pca.log')
file_handler.setLevel(logging.INFO)
file_handler.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s - %(message)s'))

logger = logging.getLogger()

logger.handlers = []
logger.addHandler(file_handler)

In [11]:
def nested_cv_extended(X, random_seeds, n_outer_folds=5, n_inner_folds=3, pca_components=[], mod_selection_score=accuracy_score, positive_class=0):
    n_estimatorss = np.arange(50, 201, 50)
    contaminations = np.linspace(0.01, 0.5, 7)
    max_featuress = np.linspace(0.1, 1.0, 5)
    scalers = [StandardScaler(), MinMaxScaler(), RobustScaler()]
    
    best_params = {'n_estimators': 0, 'contamination': 0, 'max_features': 0, 'pca': 0, 'scaler': ''}
    accuracy_scores = []
    precision_scores = []
    recall_scores = []
    f1_scores = []
    best_overall_accuracy = 0
    best_overall_params = {'n_estimators': 0, 'contamination': 0, 'max_features': 0, 'pca': 0, 'scaler': ''}

    y = X['Mezzo'].values
    y = np.array([0. if x == positive_class else 1. for x in y], dtype=float)
    
    X = X.drop(columns='Mezzo').values

    for seed in random_seeds:
        outer_cv = StratifiedKFold(n_splits=n_outer_folds, shuffle=True, random_state=seed)
        
        for outer_cv_number, (train_idx, test_idx) in enumerate(outer_cv.split(X, y)):
            X_train, X_test = X[train_idx], X[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]

            inner_cv = StratifiedKFold(n_splits=n_inner_folds, shuffle=True, random_state=seed)

            best_score = 0
            best_pca = PCA()
            best_scaler = StandardScaler()
            for inner_cv_number, (trainval_idx, valid_idx) in enumerate(inner_cv.split(X_train, y_train)):
                for params in itertools.product(n_estimatorss, contaminations, max_featuress, pca_components, scalers):
                    scaler = params[4]
                    X_trainval, X_valid = X_train[trainval_idx], X_train[valid_idx]
                    y_trainval, y_valid = y_train[trainval_idx], y_train[valid_idx]

                    X_trainval_scaled = scaler.fit_transform(X_trainval)
                    X_valid_scaled = scaler.transform(X_valid)

                    pca = PCA(n_components=params[3])
                    X_trainval_reduced = pca.fit_transform(X_trainval_scaled)
                    X_valid_reduced = pca.transform(X_valid_scaled)

                    idxs_neg = np.where(y_trainval == 1)[0]
                    X_trainval_reduced = np.delete(X_trainval_reduced, idxs_neg, axis=0)
                    y_trainval = np.delete(y_trainval, idxs_neg)

                    clf = IsolationForest(n_estimators=params[0], contamination=params[1], max_features=params[2])
                    
                    clf.fit(X_trainval_reduced)
                    
                    pred_values = clf.predict(X_valid_reduced)
                    true_values = [1 if y == 0 else -1 for y in y_valid]
                    
                    score = mod_selection_score(true_values, pred_values)
                    curr_params = {
                            'n_estimators': params[0],
                            'contamination': params[1],
                            'max_features': params[2],
                            'pca': params[3],
                            'scaler': params[4]
                    }

                    logging.info(f"inner cv number: {inner_cv_number}, {mod_selection_score.__name__}: {score}, with params: {curr_params}")
                        
                    if score > best_score:
                        best_score = score
                        best_pca = pca
                        best_scaler = scaler
                        best_params = curr_params

            idxs_neg = np.where(y_train == 1)[0]
            X_train = np.delete(X_train, idxs_neg, axis=0)
            y_train = np.delete(y_train, idxs_neg)

            X_train_scaled = best_scaler.fit_transform(X_train)
            X_test_scaled = best_scaler.transform(X_test)

            X_train_reduced = best_pca.transform(X_train_scaled)
            X_test_reduced = best_pca.transform(X_test_scaled)

            clf = IsolationForest(n_estimators=best_params['n_estimators'], contamination=best_params['contamination'], max_features=best_params['max_features'])
            clf.fit(X_train_reduced)

            pred_values = clf.predict(X_test_reduced)
            true_values = [1 if y == 0 else -1 for y in y_test]

            accuracy = accuracy_score(true_values, pred_values)
            precision = precision_score(true_values, pred_values, zero_division=0.0)
            recall = recall_score(true_values, pred_values)
            f1 = f1_score(true_values, pred_values)

            if accuracy > best_overall_accuracy:
                best_overall_accuracy = accuracy
                best_overall_params = best_params

            logging.info(f"outer cv number: {outer_cv_number}, accuracy: {accuracy}, precision: {precision}, recall: {recall}, f1: {f1} with params: {best_params}")

            accuracy_scores.append(accuracy)
            precision_scores.append(precision)
            recall_scores.append(recall)
            f1_scores.append(f1)

    return {
        'algorythm': 'IsolationForest',
        'best n_estimators': best_overall_params['n_estimators'],
        'best contamination': best_overall_params['contamination'],
        'best max_features': best_overall_params['max_features'],
        'best pca components': best_overall_params['pca'],
        'best scaler': best_overall_params['scaler'],
        'score used for model selection': mod_selection_score.__name__,
        'method used for model selection': 'nested cv',
        'accuracy mean': np.mean(accuracy_scores) * 100,
        'accuracy std': np.std(accuracy_scores) * 100,
        'precision mean': np.mean(precision_scores) * 100,
        'precision  std': np.std(precision_scores) * 100,
        'recall mean': np.mean(recall_scores) * 100,
        'recall std': np.std(recall_scores) * 100,
        'f1 mean': np.mean(f1_scores) * 100,
        'f1 std': np.std(f1_scores) * 100,
        'best overall accuracy': best_overall_accuracy * 100,
        'class': positive_class
    }

In [12]:
scores_df = add_record(scores_df, nested_cv_extended(X, random_seeds[0:1], n_outer_folds=13, n_inner_folds=6, pca_components=[100, 88, 75], mod_selection_score=accuracy_score, positive_class=0))
scores_df

,algorythm,best n_estimators,best contamination,best max_features,best pca components,best scaler,score used for model selection,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,f1 mean,f1 std,best overall accuracy,class
0,IsolationForest,50,0.500000,1.000,35,StandardScaler(),accuracy_score,nested cv,63.846154,7.133553,62.919677,5.075405,80.000000,8.329931,70.365347,6.093081,76.923077,0
1,IsolationForest,50,0.500000,1.000,28,RobustScaler(),f1_score,nested cv,66.923077,5.217177,64.777274,5.837249,90.000000,10.690450,74.557812,2.470739,73.076923,0
2,IsolationForest,100,0.091667,0.325,100,MinMaxScaler(),accuracy_score,nested cv,63.076923,14.876215,67.399267,19.805760,65.641026,23.329106,64.197896,17.428678,80.000000,0


In [13]:
file_path = 'if_exp5_df.pickle'

with open(file_path, 'wb') as file:
    pickle.dump(scores_df, file)

In [14]:
scores_df = add_record(scores_df, nested_cv_extended(X, random_seeds[0:1], n_outer_folds=13, n_inner_folds=6, pca_components=[100, 88, 75], mod_selection_score=f1_score, positive_class=0))
scores_df

,algorythm,best n_estimators,best contamination,best max_features,best pca components,best scaler,score used for model selection,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,f1 mean,f1 std,best overall accuracy,class
0,IsolationForest,50,0.500000,1.000,35,StandardScaler(),accuracy_score,nested cv,63.846154,7.133553,62.919677,5.075405,80.000000,8.329931,70.365347,6.093081,76.923077,0
1,IsolationForest,50,0.500000,1.000,28,RobustScaler(),f1_score,nested cv,66.923077,5.217177,64.777274,5.837249,90.000000,10.690450,74.557812,2.470739,73.076923,0
2,IsolationForest,100,0.091667,0.325,100,MinMaxScaler(),accuracy_score,nested cv,63.076923,14.876215,67.399267,19.805760,65.641026,23.329106,64.197896,17.428678,80.000000,0
3,IsolationForest,200,0.255000,0.325,88,RobustScaler(),f1_score,nested cv,62.307692,10.490909,65.256410,12.373511,75.897436,23.136675,67.101403,11.661966,80.000000,0


In [15]:
file_path = 'if_exp5_df.pickle'

with open(file_path, 'wb') as file:
    pickle.dump(scores_df, file)